In [ ]:
import os

# Set the target folder
yolox_dir = "./YOLOX"
os.chdir(yolox_dir)
print(f"⚠️ Changed working directory to: {os.getcwd()}")

In [ ]:
import os
import glob
import subprocess

def setup_complete_vs_environment():
    """
    Complete Visual Studio environment setup for PyTorch C++ extensions
    """
    vs_path = r"C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools"
    
    print("Setting up complete Visual Studio environment...")
    
    # 1. Find Windows SDK
    sdk_bases = [
        r"C:\Program Files (x86)\Windows Kits\10",
        r"C:\Program Files\Windows Kits\10"
    ]
    
    include_paths = []
    lib_paths = []
    
    for base in sdk_bases:
        if os.path.exists(base):
            # Find SDK versions
            include_base = os.path.join(base, "Include")
            lib_base = os.path.join(base, "Lib")
            
            if os.path.exists(include_base):
                versions = [d for d in os.listdir(include_base) if d.startswith("10.")]
                if versions:
                    latest_version = max(versions)
                    sdk_include = os.path.join(include_base, latest_version)
                    sdk_lib = os.path.join(lib_base, latest_version)
                    
                    # Add SDK include paths
                    include_paths.extend([
                        os.path.join(sdk_include, "ucrt"),
                        os.path.join(sdk_include, "um"),
                        os.path.join(sdk_include, "shared"),
                        os.path.join(sdk_include, "winrt"),
                        os.path.join(sdk_include, "cppwinrt"),
                    ])
                    
                    # Add SDK library paths
                    lib_paths.extend([
                        os.path.join(sdk_lib, "ucrt", "x64"),
                        os.path.join(sdk_lib, "um", "x64"),
                    ])
                    
                    print(f"Found Windows SDK {latest_version}")
                    break
    
    # 2. Find MSVC tools
    vc_tools_path = os.path.join(vs_path, "VC", "Tools", "MSVC")
    if os.path.exists(vc_tools_path):
        versions = os.listdir(vc_tools_path)
        if versions:
            latest_version = max(versions)
            msvc_base = os.path.join(vc_tools_path, latest_version)
            
            # Add MSVC include paths
            include_paths.extend([
                os.path.join(msvc_base, "include"),
                os.path.join(msvc_base, "atlmfc", "include"),
            ])
            
            # Add MSVC library paths
            lib_paths.extend([
                os.path.join(msvc_base, "lib", "x64"),
                os.path.join(msvc_base, "atlmfc", "lib", "x64"),
            ])
            
            # Add compiler to PATH
            bin_path = os.path.join(msvc_base, "bin", "Hostx64", "x64")
            current_path = os.environ.get('PATH', '')
            os.environ['PATH'] = f"{bin_path};{current_path}"
            
            print(f"Found MSVC {latest_version}")
    
    # 3. Set environment variables
    # Include paths
    current_include = os.environ.get('INCLUDE', '')
    new_include = ';'.join(include_paths + ([current_include] if current_include else []))
    os.environ['INCLUDE'] = new_include
    
    # Library paths
    current_lib = os.environ.get('LIB', '')
    new_lib = ';'.join(lib_paths + ([current_lib] if current_lib else []))
    os.environ['LIB'] = new_lib
    
    # Essential VS environment variables
    os.environ.update({
        'DISTUTILS_USE_SDK': '1',
        'MSSdk': '1',
        'VS160COMNTOOLS': f"{vs_path}\\Common7\\Tools\\",
        'VCINSTALLDIR': f"{vs_path}\\VC\\",
        'WindowsSDKDir': sdk_bases[0] + "\\" if os.path.exists(sdk_bases[0]) else "",
        'PLATFORM': 'x64',
        'PROCESSOR_ARCHITECTURE': 'AMD64',
    })
    
    print(f"Set up {len(include_paths)} include paths")
    print(f"Set up {len(lib_paths)} library paths")
    
    # 4. Verify key libraries exist
    key_libs = ['kernel32.lib', 'msvcprt.lib', 'msvcrt.lib', 'oldnames.lib']
    found_libs = {}
    
    for lib_name in key_libs:
        for lib_path in lib_paths:
            lib_file = os.path.join(lib_path, lib_name)
            if os.path.exists(lib_file):
                found_libs[lib_name] = lib_file
                break
    
    print(f"\nFound libraries: {list(found_libs.keys())}")
    missing_libs = set(key_libs) - set(found_libs.keys())
    if missing_libs:
        print(f"Missing libraries: {list(missing_libs)}")
        
        # Try to find them in other locations
        print("Searching for missing libraries...")
        for lib_name in missing_libs:
            for lib_path in lib_paths:
                if os.path.exists(lib_path):
                    all_libs = [f for f in os.listdir(lib_path) if f.endswith('.lib')]
                    similar = [lib for lib in all_libs if lib_name.split('.')[0] in lib]
                    if similar:
                        print(f"  In {lib_path}: found similar {similar[:3]}")
    
    return len(missing_libs) == 0

def clean_torch_cache():
    """Clean PyTorch extension cache to force recompilation"""
    cache_path = os.path.expanduser("~/.cache/torch_extensions")
    if os.path.exists(cache_path):
        import shutil
        shutil.rmtree(cache_path)
        print("Cleaned PyTorch extensions cache")
    
    # Also clean the specific cache location
    local_cache = r"C:\Users\aarnaizl\AppData\Local\torch_extensions"
    if os.path.exists(local_cache):
        import shutil
        shutil.rmtree(local_cache)
        print("Cleaned local PyTorch extensions cache")

# Run the setup
print("=== Setting up Visual Studio Environment ===")
success = setup_complete_vs_environment()

if success:
    print("\n✅ Environment setup complete!")
    print("🗑️ Cleaning PyTorch cache for fresh compilation...")
    clean_torch_cache()
    print("\n🚀 Ready to run training with fast_cocoeval!")
else:
    print("\n⚠️ Some libraries are missing. You may need to:")
    print("Check Visual Studio Build Tools is installed with the required modules (see README.md)")
    
print("\nYou can now run your training command.")

In [ ]:
import os
import cv2
import torch
import numpy as np
from yolox.exp import get_exp
from yolox.utils import postprocess, vis

# === CONFIG ===
model_path = r'D:\Ainhoa\traffic_signs_data\models\YOLOX_outputs_dfg_m\yolox_m\best_ckpt.pth'
video_path = "videos/dfg_traffic_sign_demo.mp4"
output_dir = "videos"
os.makedirs(output_dir, exist_ok=True)
video_filename = os.path.basename(video_path)
output_video_path = os.path.join(output_dir, f"detected_{video_filename}")

# YOLOX model configuration
exp_file = None  # Path to experiment file, or None for default
model_name = "yolox-m"  # Options: yolox-nano, yolox-tiny, yolox-s, yolox-m, yolox-l, yolox-x
input_size = (1024, 1024)  # Model input size
conf_thresh = 0.7
nms_thresh = 0.65

# Load class names
class_names_txt = "classes.txt"
with open(class_names_txt, 'r') as f:
    class_names = [line.strip() for line in f if line.strip()]

# === SETUP YOLOX MODEL ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get experiment configuration
if exp_file is None:
    exp = get_exp(None, model_name)
else:
    exp = get_exp(exp_file, None)

# Update experiment config
exp.num_classes = len(class_names)
exp.test_conf = conf_thresh
exp.nmsthre = nms_thresh
exp.test_size = input_size

# Build model
model = exp.get_model()
model.to(device)
model.eval()

# Load weights
ckpt = torch.load(model_path, map_location=device)
model.load_state_dict(ckpt["model"])
print(f"[INFO] Loaded YOLOX model from {model_path}")

def preprocess(img, input_size):
    """Preprocess image for YOLOX"""
    if len(img.shape) == 3:
        padded_img = np.ones((input_size[0], input_size[1], 3), dtype=np.uint8) * 114
    else:
        padded_img = np.ones(input_size, dtype=np.uint8) * 114

    r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
    resized_img = cv2.resize(
        img,
        (int(img.shape[1] * r), int(img.shape[0] * r)),
        interpolation=cv2.INTER_LINEAR,
    ).astype(np.uint8)
    
    padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img
    padded_img = padded_img.transpose((2, 0, 1))
    padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)
    
    return padded_img, r

def draw_detections(img, outputs, ratio, class_names):
    """Draw bounding boxes and labels on image"""
    if outputs is None:
        return img
    
    bboxes = outputs[:, 0:4]
    cls = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    # Scale bboxes back to original image size
    bboxes /= ratio
    
    for i, box in enumerate(bboxes):
        x1, y1, x2, y2 = box.astype(int)
        cls_id = int(cls[i])
        score = scores[i]
        
        if cls_id < len(class_names):
            class_name = class_names[cls_id]
        else:
            class_name = f"class_{cls_id}"
        
        # Draw green bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw label
        label_text = f"{class_name} {score:.2f}"
        cv2.putText(img, label_text, (x1, y1 - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, 
                   lineType=cv2.LINE_AA)
    
    return img

# === READ VIDEO ===
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# === SAVE VIDEO SETUP ===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

print(f"[INFO] Processing video: {video_path}")
print(f"[INFO] Saving output to: {output_video_path}")
print(f"[INFO] Using device: {device}")

frame_count = 0
with torch.no_grad():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # === PREPROCESS ===
        img, ratio = preprocess(frame, input_size)
        img = torch.from_numpy(img).unsqueeze(0).float().to(device)

        # === INFERENCE ===
        outputs = model(img)
        outputs = postprocess(outputs, exp.num_classes, conf_thresh, nms_thresh)[0]

        # === DRAW DETECTIONS ===
        if outputs is not None:
            frame = draw_detections(frame, outputs.cpu().numpy(), ratio, class_names)

        # === WRITE FRAME ===
        out.write(frame)
        frame_count += 1
        
        if frame_count % 100 == 0:
            print(f"[INFO] Processed {frame_count} frames")

cap.release()
out.release()
print(f"[✅] Done. Processed {frame_count} frames.")

In [ ]:
# import os
# import cv2
# import torch
# import numpy as np
# import json
# from yolox.exp import get_exp
# from yolox.utils import postprocess, vis

# # === CONFIG ===
# model_path = r'D:\Ainhoa\traffic_signs_data\models\YOLOX_outputs_dfg_m_single_class\yolox_m\best_ckpt.pth'
# video_path = "videos/dfg_traffic_sign_demo.mp4"
# output_dir = "videos"
# os.makedirs(output_dir, exist_ok=True)
# video_filename = os.path.basename(video_path)
# output_video_path = os.path.join(output_dir, f"detected_{video_filename}")
# detections_json_path = os.path.join(output_dir, f"detections_{video_filename.split('.')[0]}.json")

# # YOLOX model configuration
# exp_file = None  # Path to experiment file, or None for default
# model_name = "yolox-m"  # Options: yolox-nano, yolox-tiny, yolox-s, yolox-m, yolox-l, yolox-x
# input_size = (1024, 1024)  # Model input size
# conf_thresh = 0.7
# nms_thresh = 0.65

# # Load class names
# class_names_txt = "class_names.txt"
# with open(class_names_txt, 'r') as f:
#     class_names = [line.strip() for line in f if line.strip()]

# # === SETUP YOLOX MODEL ===
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Get experiment configuration
# if exp_file is None:
#     exp = get_exp(None, model_name)
# else:
#     exp = get_exp(exp_file, None)

# # Update experiment config
# exp.num_classes = len(class_names)
# exp.test_conf = conf_thresh
# exp.nmsthre = nms_thresh
# exp.test_size = input_size

# # Build model
# model = exp.get_model()
# model.to(device)
# model.eval()

# # Load weights
# ckpt = torch.load(model_path, map_location=device)
# model.load_state_dict(ckpt["model"])
# print(f"[INFO] Loaded YOLOX model from {model_path}")

# def preprocess(img, input_size):
#     """Preprocess image for YOLOX"""
#     if len(img.shape) == 3:
#         padded_img = np.ones((input_size[0], input_size[1], 3), dtype=np.uint8) * 114
#     else:
#         padded_img = np.ones(input_size, dtype=np.uint8) * 114

#     r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
#     resized_img = cv2.resize(
#         img,
#         (int(img.shape[1] * r), int(img.shape[0] * r)),
#         interpolation=cv2.INTER_LINEAR,
#     ).astype(np.uint8)
    
#     padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img
#     padded_img = padded_img.transpose((2, 0, 1))
#     padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)
    
#     return padded_img, r

# def extract_detections(outputs, ratio, class_names, frame_number):
#     """Extract detection information for JSON export"""
#     detections = []
    
#     if outputs is None:
#         return detections
    
#     bboxes = outputs[:, 0:4]
#     cls = outputs[:, 6]
#     scores = outputs[:, 4] * outputs[:, 5]
    
#     # Scale bboxes back to original image size
#     bboxes /= ratio
    
#     for i, box in enumerate(bboxes):
#         x1, y1, x2, y2 = box.astype(int)
#         cls_id = int(cls[i])
#         score = float(scores[i])
        
#         if cls_id < len(class_names):
#             class_name = class_names[cls_id]
#         else:
#             class_name = f"class_{cls_id}"
        
#         detection = {
#             "frame": frame_number,
#             "bbox": [int(x1), int(y1), int(x2), int(y2)],
#             "class_id": cls_id,
#             "class_name": class_name,
#             "confidence": score,
#             "detection_id": f"frame_{frame_number}_det_{i}"
#         }
#         detections.append(detection)
    
#     return detections

# def draw_detections(img, outputs, ratio, class_names):
#     """Draw bounding boxes and labels on image"""
#     if outputs is None:
#         return img
    
#     bboxes = outputs[:, 0:4]
#     cls = outputs[:, 6]
#     scores = outputs[:, 4] * outputs[:, 5]
    
#     # Scale bboxes back to original image size
#     bboxes /= ratio
    
#     for i, box in enumerate(bboxes):
#         x1, y1, x2, y2 = box.astype(int)
#         cls_id = int(cls[i])
#         score = scores[i]
        
#         if cls_id < len(class_names):
#             class_name = class_names[cls_id]
#         else:
#             class_name = f"class_{cls_id}"
        
#         # Draw green bounding box
#         cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
#         # Draw label
#         label_text = f"{class_name} {score:.2f}"
#         cv2.putText(img, label_text, (x1, y1 - 10), 
#                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, 
#                    lineType=cv2.LINE_AA)
    
#     return img

# def save_detection_crops(frame, outputs, ratio, frame_number, crops_dir):
#     """Save cropped images of detections for classification"""
#     if outputs is None:
#         return []
    
#     os.makedirs(crops_dir, exist_ok=True)
#     crop_paths = []
    
#     bboxes = outputs[:, 0:4]
#     cls = outputs[:, 6]
#     scores = outputs[:, 4] * outputs[:, 5]
    
#     # Scale bboxes back to original image size
#     bboxes /= ratio
    
#     for i, box in enumerate(bboxes):
#         x1, y1, x2, y2 = box.astype(int)
        
#         # Ensure coordinates are within frame bounds
#         h, w = frame.shape[:2]
#         x1, y1 = max(0, x1), max(0, y1)
#         x2, y2 = min(w, x2), min(h, y2)
        
#         # Crop the detection
#         crop = frame[y1:y2, x1:x2]
        
#         # Save crop
#         crop_filename = f"frame_{frame_number}_det_{i}.jpg"
#         crop_path = os.path.join(crops_dir, crop_filename)
#         cv2.imwrite(crop_path, crop)
#         crop_paths.append(crop_path)
    
#     return crop_paths

# # === READ VIDEO ===
# cap = cv2.VideoCapture(video_path)
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = cap.get(cv2.CAP_PROP_FPS)

# # === SAVE VIDEO SETUP ===
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# # Setup for saving detection crops
# crops_dir = os.path.join(output_dir, "detection_crops")

# print(f"[INFO] Processing video: {video_path}")
# print(f"[INFO] Saving output to: {output_video_path}")
# print(f"[INFO] Saving detections to: {detections_json_path}")
# print(f"[INFO] Saving detection crops to: {crops_dir}")
# print(f"[INFO] Using device: {device}")

# frame_count = 0
# all_detections = []

# # Video metadata
# video_metadata = {
#     "video_path": video_path,
#     "video_filename": video_filename,
#     "width": width,
#     "height": height,
#     "fps": fps,
#     "model_config": {
#         "model_name": model_name,
#         "input_size": input_size,
#         "conf_thresh": conf_thresh,
#         "nms_thresh": nms_thresh
#     },
#     "class_names": class_names
# }

# with torch.no_grad():
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # === PREPROCESS ===
#         img, ratio = preprocess(frame, input_size)
#         img = torch.from_numpy(img).unsqueeze(0).float().to(device)

#         # === INFERENCE ===
#         outputs = model(img)
#         outputs = postprocess(outputs, exp.num_classes, conf_thresh, nms_thresh)[0]

#         # === EXTRACT DETECTIONS FOR JSON ===
#         frame_detections = extract_detections(
#             outputs.cpu().numpy() if outputs is not None else None, 
#             ratio, class_names, frame_count
#         )
#         all_detections.extend(frame_detections)

#         # === SAVE DETECTION CROPS ===
#         if outputs is not None:
#             crop_paths = save_detection_crops(frame, outputs.cpu().numpy(), ratio, frame_count, crops_dir)
#             # Add crop paths to detections
#             for det, crop_path in zip(frame_detections, crop_paths):
#                 det["crop_path"] = crop_path

#         # === DRAW DETECTIONS ===
#         if outputs is not None:
#             frame = draw_detections(frame, outputs.cpu().numpy(), ratio, class_names)

#         # === WRITE FRAME ===
#         out.write(frame)
#         frame_count += 1
        
#         if frame_count % 100 == 0:
#             print(f"[INFO] Processed {frame_count} frames")

# cap.release()
# out.release()

# # === SAVE DETECTIONS TO JSON ===
# detection_data = {
#     "video_metadata": video_metadata,
#     "total_frames": frame_count,
#     "total_detections": len(all_detections),
#     "detections": all_detections
# }

# with open(detections_json_path, 'w') as f:
#     json.dump(detection_data, f, indent=2)

# print(f"[✅] Done. Processed {frame_count} frames.")
# print(f"[✅] Saved {len(all_detections)} detections to {detections_json_path}")
# print(f"[✅] Saved detection crops to {crops_dir}")